In [2]:
# !wget https://huggingface.co/datasets/mesolitica/translated-MNLI/resolve/main/translated-mnli-train.jsonl
# !wget https://huggingface.co/datasets/mesolitica/translated-MNLI/resolve/main/translated-mnli-dev_matched.jsonl

In [2]:
# !wget https://huggingface.co/datasets/mesolitica/translated-MNLI/resolve/main/anli.translated.jsonl

In [3]:
from tqdm import tqdm
from glob import glob
import json
import random

random.random()

0.16337827077471856

In [4]:
labels = ['contradiction', 'entailment']

In [5]:
!head -n 1 translated-mnli-train.jsonl

{"annotator_labels": ["neutral"], "genre": "government", "gold_label": "neutral", "pairID": "31193n", "promptID": "31193", "sentence1": "Conceptually cream skimming has two basic dimensions - product and geography.", "sentence1_binary_parse": "( ( Conceptually ( cream skimming ) ) ( ( has ( ( ( two ( basic dimensions ) ) - ) ( ( product and ) geography ) ) ) . ) )", "sentence1_parse": "(ROOT (S (NP (JJ Conceptually) (NN cream) (NN skimming)) (VP (VBZ has) (NP (NP (CD two) (JJ basic) (NNS dimensions)) (: -) (NP (NN product) (CC and) (NN geography)))) (. .)))", "sentence2": "Product and geography are what make cream skimming work. ", "sentence2_binary_parse": "( ( ( Product and ) geography ) ( ( are ( what ( make ( cream ( skimming work ) ) ) ) ) . ) )", "sentence2_parse": "(ROOT (S (NP (NN Product) (CC and) (NN geography)) (VP (VBP are) (SBAR (WHNP (WP what)) (S (VP (VBP make) (NP (NP (NN cream)) (VP (VBG skimming) (NP (NN work)))))))) (. .)))", "translate": ["Skim krim konseptual mempu

In [6]:
!head -n 1 anli.translated.jsonl

{"uid": "2093cfb3-a15f-4282-81e3-0cb793ffd0d7", "premise": "TOKYO, Dec 18 (Reuters) - Japan\u2019s Shionogi & Co said on Tuesday that it has applied to health regulators in the United States, Canada and Europe for approval of its HIV drug Dolutegravir. Shionogi developed Dolutegravir with a Viiv Healthcare, an AIDS drug joint venture between GlaxoSmithKline and Pfizer, in exchange for its rights to the drug.", "hypothesis": "The article was written on December 18th.", "label": 0, "reason": "TOKYO, Dec 18 (Reuters) is when the article was written as it states in the first words of the sentence", "premise_ms": "TOKYO, 18 Dis (Reuters) - Shionogi & Co Jepun berkata pada hari Selasa bahawa ia telah memohon kepada pengawal selia kesihatan di Amerika Syarikat, Kanada dan Eropah untuk kelulusan ubat HIVnya Dolutegravir. Shionogi membangunkan Dolutegravir dengan Viiv Healthcare, sebuah usaha sama ubat AIDS antara GlaxoSmithKline dan Pfizer, sebagai pertukaran untuk haknya terhadap ubat tersebu

In [8]:
data

{'uid': '2093cfb3-a15f-4282-81e3-0cb793ffd0d7',
 'premise': 'TOKYO, Dec 18 (Reuters) - Japan’s Shionogi & Co said on Tuesday that it has applied to health regulators in the United States, Canada and Europe for approval of its HIV drug Dolutegravir. Shionogi developed Dolutegravir with a Viiv Healthcare, an AIDS drug joint venture between GlaxoSmithKline and Pfizer, in exchange for its rights to the drug.',
 'hypothesis': 'The article was written on December 18th.',
 'label': 0,
 'reason': 'TOKYO, Dec 18 (Reuters) is when the article was written as it states in the first words of the sentence',
 'premise_ms': 'TOKYO, 18 Dis (Reuters) - Shionogi & Co Jepun berkata pada hari Selasa bahawa ia telah memohon kepada pengawal selia kesihatan di Amerika Syarikat, Kanada dan Eropah untuk kelulusan ubat HIVnya Dolutegravir. Shionogi membangunkan Dolutegravir dengan Viiv Healthcare, sebuah usaha sama ubat AIDS antara GlaxoSmithKline dan Pfizer, sebagai pertukaran untuk haknya terhadap ubat tersebu

In [9]:
anli = {
    0: 1,
    2: 0,
}

In [15]:
!head -n 1 train.json

{"src": "ayat1: TOKYO, Dec 18 (Reuters) - Japan\u2019s Shionogi & Co said on Tuesday that it has applied to health regulators in the United States, Canada and Europe for approval of its HIV drug Dolutegravir. Shionogi developed Dolutegravir with a Viiv Healthcare, an AIDS drug joint venture between GlaxoSmithKline and Pfizer, in exchange for its rights to the drug. ayat2: The article was written on December 18th.", "label": 1}


In [21]:
with open('train.json', 'w') as fopen_jsonl:
    with open('anli.translated.jsonl') as fopen:
        for l in tqdm(fopen):
            data = json.loads(l)
            label = data['label']
            label = anli.get(label)
            if label is None:
                continue
                
            sent1 = data['premise'].strip()
            sent2 = data['hypothesis'].strip()
            
            sent1_ms = data['premise_ms'].strip()
            sent2_ms = data['hypothesis_ms'].strip()
            
            left = f'ayat1: {sent1} ayat2: {sent2}'
            d = {"src": left, "label": label}
            fopen_jsonl.write(f'{json.dumps(d)}\n')
            
            left = f'ayat1: {sent1_ms} ayat2: {sent2_ms}'
            d = {"src": left, "label": label}
            fopen_jsonl.write(f'{json.dumps(d)}\n')
            
            if random.random() > 0.7:
            
                left = f'ayat1: {sent1} ayat2: {sent2_ms}'
                d = {"src": left, "label": label}
                fopen_jsonl.write(f'{json.dumps(d)}\n')
            
            if random.random() > 0.7:
            
                left = f'ayat1: {sent1_ms} ayat2: {sent2}'
                d = {"src": left, "label": label}
                fopen_jsonl.write(f'{json.dumps(d)}\n')
            
    with open('translated-mnli-train.jsonl') as fopen:
        for l in tqdm(fopen):
            data = json.loads(l)
            label = data['gold_label']
            if label not in labels:
                continue
            if label == '-':
                continue
            
            label = labels.index(label)
            sent1 = data['sentence1'].strip()
            sent2 = data['sentence2'].strip()
            
            sent1_ms = data['translate'][0].strip()
            sent2_ms = data['translate'][1].strip()
            
            left = f'ayat1: {sent1} ayat2: {sent2}'
            d = {"src": left, "label": label}
            fopen_jsonl.write(f'{json.dumps(d)}\n')
            
            left = f'ayat1: {sent1_ms} ayat2: {sent2_ms}'
            d = {"src": left, "label": label}
            fopen_jsonl.write(f'{json.dumps(d)}\n')
            
            if random.random() > 0.7:
            
                left = f'ayat1: {sent1} ayat2: {sent2_ms}'
                d = {"src": left, "label": label}
                fopen_jsonl.write(f'{json.dumps(d)}\n')
                
            if random.random() > 0.7:
            
                left = f'ayat1: {sent1_ms} ayat2: {sent2}'
                d = {"src": left, "label": label}
                fopen_jsonl.write(f'{json.dumps(d)}\n')
    
    with open('sentiment.json') as fopen:
        data = json.load(fopen)
        for i in range(len(data['X1'])):
            l = data['X1'][i]
            r = data['X2'][i]
            left = f'ayat1: {l} ayat2: {r}'
            d = {"src": left, "label": data['Y'][i]}
            fopen_jsonl.write(f'{json.dumps(d)}\n')
            
    with open('topics.json') as fopen:
        data = json.load(fopen)
        for i in range(len(data['X1'])):
            l = data['X1'][i]
            r = data['X2'][i]
            left = f'ayat1: {l} ayat2: {r}'
            d = {"src": left, "label": data['Y'][i]}
            fopen_jsonl.write(f'{json.dumps(d)}\n')

100459it [00:00, 137008.56it/s]
392702it [00:03, 124487.94it/s]


In [22]:
!wc -l train.json

1051464 train.json


In [23]:
!shuf train.json > shuffled-train.json

In [24]:
with open('test.json', 'w') as fopen_jsonl:
    with open('translated-mnli-dev_matched.jsonl') as fopen:
        for l in tqdm(fopen):
            data = json.loads(l)
            label = data['gold_label']
            if label not in labels:
                continue
            if label == '-':
                continue
            
            label = labels.index(label)
                
            sent1 = data['sentence1'].strip()
            sent2 = data['sentence2'].strip()
            
            sent1_ms = data['translate'][0].strip()
            sent2_ms = data['translate'][1].strip()
            
            left = f'ayat1: {sent1} ayat2: {sent2}'
            d = {"src": left, "label": label}
            fopen_jsonl.write(f'{json.dumps(d)}\n')
            
            left = f'ayat1: {sent1} ayat2: {sent2_ms}'
            d = {"src": left, "label": label}
            fopen_jsonl.write(f'{json.dumps(d)}\n')
            
            left = f'ayat1: {sent1_ms} ayat2: {sent2_ms}'
            d = {"src": left, "label": label}
            fopen_jsonl.write(f'{json.dumps(d)}\n')
            
            left = f'ayat1: {sent1_ms} ayat2: {sent2}'
            d = {"src": left, "label": label}
            fopen_jsonl.write(f'{json.dumps(d)}\n')

10000it [00:00, 71898.45it/s]


In [25]:
!shuf test.json > shuffled-test.json

In [26]:
!head -n 1000 shuffled-test.json > test-1k.json

In [27]:
!wc -l shuffled-test.json

26768 shuffled-test.json


In [28]:
!head -n 10 shuffled-test.json

{"src": "ayat1: and it just depends on how bad that person is ayat2: Ia bergantung kepada keadaan anjing.", "label": 0}
{"src": "ayat1: Smart men make good thieves, as long as they're desperate. ayat2: It is best for a thief to be dumb.", "label": 0}
{"src": "ayat1: i also use my PC to emulate a mainframe terminal for our IBM mainframe and also to emulate a deck terminal for our deck machine ayat2: PC saya tidak pernah digunakan untuk meniru apa-apa.", "label": 0}
{"src": "ayat1: Dia mempunyai pencerobohan menyedihkan seorang isteri atau ibu - kepada Bunt tidak ada perbezaan. ayat2: Bunt didn't differentiate from a wife's or mother's agression.", "label": 1}
{"src": "ayat1: Then it occurred to me that the criminal standard was a low one. ayat2: Saya sedar bahawa penjenayah mempunyai standard yang sangat tinggi.", "label": 0}
{"src": "ayat1: um-hum betul lakukan di mana anda berada di negeri apa ayat2: Berapa batu yang anda jalankan?", "label": 0}
{"src": "ayat1: Ia bernilai lawatan, ji

In [2]:
from typing import List, Dict, Optional
from pydantic import BaseModel, Field

In [3]:
class FunctionCall(BaseModel):
    name: Optional[str] = None
    arguments: str

In [6]:
form = FunctionCall(arguments = 'a')

In [9]:
getattr(form, 'a', False)

False